******OPEN DATAFRAME WITH POLARS******

In [1]:
#Parallelism
import polars as pl

#Import modules
import matplotlib.pyplot as plt
%matplotlib inline

#handling data
import pandas as pd
import numpy as np
import json
import networkx as nx
from datetime import datetime

import random

In [2]:
#load data (********CREATI con il notebook Apple_wsb_filter)
data_sub1=pl.read_ndjson(r'C:\Tempor\reddit\network\df_sub_wsb_apple_1.json')
data_com1=pl.read_ndjson(r'C:\Tempor\reddit\network\df_com_wsb_1.json')

data_sub2=pl.read_ndjson(r'C:\Tempor\reddit\network\df_sub_wsb_apple_2.json')
data_com2=pl.read_ndjson(r'C:\Tempor\reddit\network\df_com_wsb_2.json')

data_sub3=pl.read_ndjson(r'C:\Tempor\reddit\network\df_sub_wsb_apple_3.json')
data_com3=pl.read_ndjson(r'C:\Tempor\reddit\network\df_com_wsb_3.json')

data_sub4=pl.read_ndjson(r'C:\Tempor\reddit\network\df_sub_wsb_apple_4.json')
data_com4=pl.read_ndjson(r'C:\Tempor\reddit\network\df_com_wsb_4.json')

data_sub5=pl.read_ndjson(r'C:\Tempor\reddit\network\df_sub_wsb_apple_5.json')
data_com5=pl.read_ndjson(r'C:\Tempor\reddit\network\df_com_wsb_5.json')

data_sub=pl.concat([data_sub1,data_sub2,data_sub3,data_sub4,data_sub5])
data_com=pl.concat([data_com1,data_com2,data_com3,data_com4,data_com5])

#select columns
df_sub_wsb_apple=data_sub.select(["id","author","body","name","score","num_comments","subreddit","subreddit_id","from","from_id","created_utc"])
df_com_wsb=data_com.select(["id","author","body","name","parent_id","link_id","subreddit","subreddit_id","created_utc"])


#Dataframe length
print('la lunghezza del dataframe di submissions è ' + str(len(df_sub_wsb_apple)))
print('la lunghezza del dataframe di comments è ' + str(len(df_com_wsb)))


la lunghezza del dataframe di submissions è 1339
la lunghezza del dataframe di comments è 1581213


In [4]:
subreddit_list=("wallstreetbets","finance","investing","Frugal","SecurityAnalysis",
                "RobinHood","InvestmentClub","Forex","options","CryptoCurrency",
                "StockMarket","economy","Economics","GlobalMarkets","dividends",
                "Daytrading","algotrading","pennystocks","ValueInvesting","Bogleheads",
                "RobinHoodPennyStocks","RealDayTrading","investingforbeginners","business",
                "Flipping","Entrepreneur","wallstreet","wallstreetdd","MiddleClassFinance",
                "beermoney","DropShipping101","smallbusiness","churning","giftcardexchange",
                "personalfinance","FinancialPlanning","CRedit","povertyfinance",
                "financialindipendence","leanfire","flatFIRE","StudentLoans")

In [5]:
# sub_business=df_sub_wsb_apple.filter(df_sub_wsb_apple["subreddit"]=="business").select(["author"]).unique()
# com_business=df_com_wsb.filter(df_com_wsb["subreddit"]=="business").select(["author"]).unique()
# business=pl.concat([sub_business,com_business])
# sub_business_list=sub_business["author"].to_list()
# business_nodes=business.unique()["author"].to_list()
# g=nx.Graph()
# for node in sub_business_list:
#     g.add_edges_from([(node,other) for other in business_nodes])

# g.remove_node('[deleted]')

***NETWORK CREATION***

In [3]:
import networkx as nx

In [ ]:
#Network creation
G=nx.Graph()
G.add_nodes_from(df_sub_wsb_apple['author'])

# len(G.nodes)
# df_com_wsb['parent_id'][2]


authors=df_sub_wsb_apple['name'].to_list()
Bool_vector=np.array(df_com_wsb['link_id'].str.contains('|'.join(authors)))
df_com_wsb_apple=df_com_wsb.filter(Bool_vector)
# len(df_com_wsb_apple)

#Adding edges between Submission's author and repliers
for i in range(int(len(df_sub_wsb_apple))):
    df_links=df_sub_wsb_apple.filter(df_sub_wsb_apple['name']==df_com_wsb_apple['link_id'][i]) 
    for j in range(len(df_links)):
        if (df_com_wsb_apple['author'][i]!= df_links['author'][j]):
            G.add_edge(df_com_wsb_apple['author'][i], df_links['author'][j], weight=1)
            
#Adding edges between repliers
for i in range(int(len(df_com_wsb_apple))):
    df_links=df_com_wsb_apple.filter(df_com_wsb_apple['name']==df_com_wsb_apple['parent_id'][i])
    for j in range(len(df_links)):
        if (df_com_wsb_apple['author'][i]!= df_links['author'][j]):
            G.add_edge(df_com_wsb_apple['author'][i], df_links['author'][j], weight=1)

#Adding edges between users of the same subreddit, with weight=0.5
# for subreddit in subreddit_list:
#     sub_business=df_sub_wsb_apple.filter(df_sub_wsb_apple["subreddit"]==subreddit).select(["author"]).unique()
#     com_business=df_com_wsb.filter(df_com_wsb["subreddit"]==subreddit).select(["author"]).unique()
#     business=pl.concat([sub_business,com_business])
#     sub_business_list=sub_business["author"].to_list()
#     business_nodes=business.unique()["author"].to_list()
#     for node in sub_business_list:
#         G.add_edges_from([(node,other) for other in business_nodes], weight=0.01)
         
#Remove node corresponding to all the unsubscribed authors
G.remove_node('[deleted]')
        
        
print('The network with only links between authors of submissions and those who answer them directly has ' 
      + str(len(G.nodes)) + ' nodes and ' + str(len(G.edges)) + ' links')

La rete con i soli link tra autori delle submissions e chi risponde loro direttamente ha 4013 nodi e 4704 link


In [30]:
# plt.figure(figsize=(12,12))
# nx.draw_networkx(G,  with_labels=False, node_size=1)
# plt.show()

In [ ]:
print(f"There are {nx.number_connected_components(G)} connected components in the Graph")

connected_component_subgraphs = (G.subgraph(c) for c in nx.connected_components(G))
largest_subgraph = max(connected_component_subgraphs, key=len)

print(f"There are {largest_subgraph.number_of_nodes()} nodes and {largest_subgraph.number_of_edges()} \
edges present in the largest component of the Graph")

nx.write_graphml_lxml(largest_subgraph, r'C:\Tempor\reddit\network\network_5_months_all_nodes.graphml')   